# Setup Project

In [ ]:
from google.colab import auth

auth.authenticate_user()

In [ ]:
PROJECT_ID = "claimwise-ai-469715"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


In [ ]:
!bq mk --connection --location=us \
    --connection_type=CLOUD_RESOURCE test_connection

BigQuery error in mk operation: Already Exists: Connection
projects/477012212019/locations/us/connections/test_connection


In [ ]:
SERVICE_ACCT = !bq show --format=prettyjson --connection us.test_connection | grep "serviceAccountId" | cut -d '"' -f 4
SERVICE_ACCT_EMAIL = SERVICE_ACCT[-1]
print(SERVICE_ACCT_EMAIL)

bqcx-477012212019-1fi4@gcp-sa-bigquery-condel.iam.gserviceaccount.com


In [ ]:
import time

!gcloud projects add-iam-policy-binding --format=none $PROJECT_ID --member=serviceAccount:$SERVICE_ACCT_EMAIL --role='roles/bigquery.connectionUser'
!gcloud projects add-iam-policy-binding --format=none $PROJECT_ID --member=serviceAccount:$SERVICE_ACCT_EMAIL --role='roles/aiplatform.user'
!gcloud projects add-iam-policy-binding --format=none $PROJECT_ID --member=serviceAccount:$SERVICE_ACCT_EMAIL --role='roles/storage.objectViewer'

# wait 60 seconds, give IAM updates time to propagate, otherwise, following cells will fail
time.sleep(60)

Updated IAM policy for project [claimwise-ai-469715].
Updated IAM policy for project [claimwise-ai-469715].
Updated IAM policy for project [claimwise-ai-469715].


# Summarize car and accident information

In [ ]:
%%bigquery cars --project {PROJECT_ID}
SELECT
  policy_number,
  auto_make,
  auto_model,
  AI.GENERATE(
    # 'Summarize this car: make=' || CAST(auto_make AS STRING) ||
    #  ', model=' || CAST(auto_model AS STRING) ||
    #  ', incident_type=' || CAST(incident_type AS STRING) ||
    #  ', total claim=' || total_claim_amount ||
    #  ', incident location=' || incident_location || ',' || incident_city,
    'Summarize this car based on ' || auto_make || auto_model ||
    incident_type || total_claim_amount || incident_location || incident_city,
    connection_id => 'us.test_connection',
    endpoint => 'gemini-2.5-flash'
  ).result AS car_summary
FROM `claimwise-ai-469715.claimwise.insurance_claims`
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
%load_ext google.colab.data_table

In [ ]:
import pandas as pd

# remove truncation for long text fields
pd.set_option("display.max_colwidth", None)

In [ ]:
cars

,policy_number,auto_make,auto_model,car_summary
0,599174,Saab,92x,"Based on the information provided, the car is a **Saab 9-2X** that was involved in a **multi-vehicle collision** on **April 7, 2014**, at **Rock Avenue in Arlington**."
1,336614,BMW,3 Series,"Based on the information provided, the car is a **BMW 3 Series** that was involved in a **multi-vehicle collision**. This incident, identified by the number **297906092**, occurred on **5th Avenue in Arlington**."
2,444035,Saab,92x,"Based on the provided information, the car is a **Saab 9-2X**.\n\nIt was involved in a **multi-vehicle collision** at **5th Avenue in Hillsdale**. The number ""427003397"" appears to be an incident or case identifier related to the collision."
3,427199,BMW,3 Series,The car is a **BMW 3 Series** that was involved in a **multi-vehicle collision**. This incident occurred on **Francis Lane in Northbend** and is associated with the identifier/report number **457002469**.
4,633375,BMW,3 Series,"This is a **BMW 3 Series** that was involved in a **multi-vehicle collision** on **Rock Drive in Columbus**, identified by incident number **796807630**."
5,373731,BMW,3 Series,This appears to be information about a **BMW 3 Series** vehicle that was involved in a **multi-vehicle collision** at **Pine Ave in Springfield**. The incident is identified by the number **815004567**.
6,669800,Saab,92x,"The car is a **Saab 9-2X** that was involved in a **multi-vehicle collision** at **Lincoln Drive in Hillsdale**, associated with the incident number **664805352**."
7,531640,BMW,3 Series,"This refers to a **BMW 3 Series** vehicle that was involved in a **multi-vehicle collision** on **July 28, 2016**, at **28 Best Drive, Arlington**."
8,469874,Saab,92x,"This information describes a **Saab 9-2X** that was involved in a **multi-vehicle collision** in **Northbend**. The incident is identified by the number **447307756**. The term ""Solo Drive"" likely refers to the nature of the Saab's travel prior to or during its involvement in the collision."
9,868031,Saab,92x,"Based on the information provided, here's a summary of the car:\n\nThe car is a **Saab 9-2X**. It was involved in a **multi-vehicle collision** that occurred on **Apache Drive in Northbend**. The numerical sequence **970806604** likely serves as an incident identifier, with ""970806"" potentially indicating the date of the collision (August 6th, 1997)."


In [ ]:
insurance_claims = pd.read_csv('insurance_claims.csv')
insurance_claims.head()

FileNotFoundError: [Errno 2] No such file or directory: 'insurance_claims.csv'

# 3 code cells below are used only for creating GCS bucket from google drive media folder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!gsutil mb -l us-east1 gs://vehicle_media

In [ ]:
!gsutil -m cp /content/drive/MyDrive/insurance_media/* gs://vehicle_media/

# Create Reference to media files stored in Google Cloud Bucket

In [ ]:
%%bigquery --project {PROJECT_ID}

CREATE OR REPLACE EXTERNAL TABLE `claimwise-ai-469715.claimwise.vehicle_medias`
WITH CONNECTION `us.test_connection`
OPTIONS (
  object_metadata = 'SIMPLE',
  uris = ['gs://vehicle_media/*']
);

Query is running:   0%|          |

""


In [ ]:
!gsutil iam ch serviceAccount:bqcx-477012212019-1fi4@gcp-sa-bigquery-condel.iam.gserviceaccount.com:roles/storage.objectViewer gs://vehicle_media

In [ ]:
%%bigquery df --project {PROJECT_ID}

SELECT * FROM `claimwise-ai-469715.claimwise.vehicle_medias`;

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,uri,generation,content_type,size,md5_hash,updated,metadata,ref
0,gs://vehicle_media/67fdf83a892ab.image.jpg,1756136877030852,image/jpeg,22239,44583ac7551f548c003051e3b3808c10,2025-08-25 15:47:57.087000+00:00,[],{'uri': 'gs://vehicle_media/67fdf83a892ab.imag...
1,gs://vehicle_media/car-crash-1.jpeg,1756136877631205,image/jpeg,202572,cdf037d9a46c1e44d90c594cef06c42c,2025-08-25 15:47:57.687000+00:00,[],"{'uri': 'gs://vehicle_media/car-crash-1.jpeg',..."
2,gs://vehicle_media/car-crash-5.jpg,1756136877629487,image/jpeg,53572,4f673ed3fefc1e18502a8a006ff1585e,2025-08-25 15:47:57.668000+00:00,[],"{'uri': 'gs://vehicle_media/car-crash-5.jpg', ..."
3,gs://vehicle_media/car-crash-6.jpg,1756688823990929,image/jpeg,42350,8393cc6ed8d5fd2ea135586e7ea07c23,2025-09-01 01:07:04.062000+00:00,[],"{'uri': 'gs://vehicle_media/car-crash-6.jpg', ..."
4,gs://vehicle_media/car-crash.jpeg,1756136877033490,image/jpeg,10346,4271730b6d59d90b3f448d05ebd26f81,2025-08-25 15:47:57.088000+00:00,[],"{'uri': 'gs://vehicle_media/car-crash.jpeg', '..."
5,gs://vehicle_media/car-crash2.jpg,1756136877031047,image/jpeg,206916,b7372a869815288c4818f51370191e55,2025-08-25 15:47:57.068000+00:00,[],"{'uri': 'gs://vehicle_media/car-crash2.jpg', '..."


# Use Gemini to describe the car images

In [ ]:
%%bigquery df --project $PROJECT_ID

SELECT
  uri,
  CAST(
    AI.GENERATE(
      STRUCT(
        'Describe what kind of car crash is shown in this image: ', ref),
      connection_id => 'us.test_connection',
      endpoint => 'gemini-2.5-flash'
    ).result AS STRING
  ) AS crash_description
FROM `claimwise-ai-469715.claimwise.vehicle_medias`;

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,uri,crash_description
0,gs://vehicle_media/car-crash.jpeg,This image shows a **significant side impact**...
1,gs://vehicle_media/car-crash-5.jpg,The image shows a **rear-end car collision** i...
2,gs://vehicle_media/car-crash-1.jpeg,The image shows a **single-vehicle collision**...
3,gs://vehicle_media/67fdf83a892ab.image.jpg,The image depicts a **single-vehicle car crash...
4,gs://vehicle_media/car-crash-6.jpg,"Based on the image, the car crash shown is a *..."
5,gs://vehicle_media/car-crash2.jpg,"Based on the damage shown in the image, the ca..."


# Create embedding from claim information, and later use embedding to find most similar cases and estimate claim cost

In [ ]:
%%bigquery --project {PROJECT_ID}

CREATE OR REPLACE MODEL `claimwise-ai-469715.claimwise.text_embedding_model`
  REMOTE WITH CONNECTION `us.test_connection`
  OPTIONS (ENDPOINT = 'gemini-embedding-001');

In [ ]:
%%bigquery --project {PROJECT_ID}

ALTER TABLE claimwise-ai-469715.claimwise.insurance_claims
ADD COLUMN text_embedding ARRAY<FLOAT64>;

In [ ]:
%%bigquery --project {PROJECT_ID}

UPDATE `claimwise.insurance_claims` AS t
SET t.text_embedding = s.ml_generate_embedding_result
FROM (
  SELECT
    policy_number,
    ml_generate_embedding_result
  FROM
    ML.GENERATE_EMBEDDING(
      MODEL `claimwise.text_embedding_model`,
      (
        SELECT
          policy_number,
          CONCAT(auto_make, ' ', auto_model, ' ', auto_year, ' ', incident_type, ' ', collision_type) AS content
        FROM `claimwise.insurance_claims`
      ),
      STRUCT(TRUE AS flatten_json_output)
    )
) AS s
WHERE t.policy_number = s.policy_number;

In [ ]:
# This code only for revert table if we mistakenly manipulate it wrong

# %%bigquery --project {PROJECT_ID}

# CREATE OR REPLACE TABLE `claimwise.insurance_claims` AS
# SELECT *
# FROM `claimwise.insurance_claims` FOR SYSTEM_TIME AS OF TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 10 MINUTE);

In [ ]:
%%bigquery --project {PROJECT_ID}
SELECT
  policy_number,
  auto_make,
  auto_model,
  auto_year,
  incident_type,
  collision_type,
  text_embedding,
FROM `claimwise.insurance_claims`
LIMIT 5;

In [ ]:
%%bigquery text_matches --project {PROJECT_ID}

SELECT base.policy_number,	base.auto_make, base.auto_model, base.auto_year,	base.incident_type, base.collision_type, base.total_claim_amount, distance
    FROM
      VECTOR_SEARCH(
        TABLE `claimwise.insurance_claims`,
        'text_embedding',
        (
        -- GENERATE AN EMBEDDING AS A SUBQUERY
        SELECT
          ml_generate_embedding_result,
          content AS query
        FROM
          ML.GENERATE_EMBEDDING(
            MODEL `claimwise.text_embedding_model`,
            ( SELECT "a side-collision BMW X5" AS content)
          )
        ),
        top_k => 10)
ORDER BY distance ASC;

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
text_matches

,policy_number,auto_make,auto_model,auto_year,incident_type,collision_type,total_claim_amount,distance
0,296960,BMW,X5,2007,Single Vehicle Collision,Side Collision,95810,0.455273
1,616337,BMW,X5,2001,Single Vehicle Collision,Side Collision,97080,0.469114
2,351741,BMW,X5,2009,Multi-vehicle Collision,Side Collision,90600,0.519620
3,343161,BMW,M5,2011,Single Vehicle Collision,Side Collision,56300,0.564880
4,175177,BMW,X6,2005,Multi-vehicle Collision,Side Collision,42930,0.581713
5,517240,BMW,X5,2006,Single Vehicle Collision,Front Collision,63840,0.582869
6,920826,BMW,X5,2010,Single Vehicle Collision,Front Collision,75570,0.586945
7,484321,BMW,X6,2000,Multi-vehicle Collision,Side Collision,90480,0.588624
8,437442,BMW,X6,2006,Multi-vehicle Collision,Side Collision,75400,0.594151
9,717392,BMW,X5,2011,Single Vehicle Collision,Rear Collision,42000,0.596435


# Create embedding for multimodal such as images, to later detect fraud by searching similar/duplicate images in the storage

In [ ]:
%%bigquery --project {PROJECT_ID}

CREATE OR REPLACE MODEL `claimwise-ai-469715.claimwise.mm_embedding_model`
  REMOTE WITH CONNECTION `us.test_connection`
  OPTIONS (ENDPOINT = 'multimodalembedding@001');

Query is running:   0%|          |

""


In [ ]:
%%bigquery --project {PROJECT_ID}

CREATE OR REPLACE TABLE `claimwise.vehicle_medias_embeddings` AS
SELECT
  uri,
  size,
  content_type,
  metadata,
  ref,
  CAST(NULL AS ARRAY<FLOAT64>) AS mm_embedding
FROM `claimwise.vehicle_medias`;

Query is running:   0%|          |

""


In [ ]:
%%bigquery --project {PROJECT_ID}

UPDATE `claimwise.vehicle_medias_embeddings` AS t
SET t.mm_embedding = s.ml_generate_embedding_result
FROM (
  SELECT
    uri,
    ml_generate_embedding_result
  FROM ML.GENERATE_EMBEDDING(
    MODEL `claimwise.mm_embedding_model`,
    (
      SELECT
        uri,
        content_type,
        OBJ.GET_ACCESS_URL(
          OBJ.FETCH_METADATA(OBJ.MAKE_REF(uri, 'us.test_connection')),
          'r'
        ) AS content
      FROM `claimwise.vehicle_medias_embeddings`
    ),
    STRUCT(TRUE AS flatten_json_output)
  )
) AS s
WHERE t.uri = s.uri;

Query is running:   0%|          |

""


In [ ]:
%%bigquery --project {PROJECT_ID}

SELECT *
FROM `claimwise.vehicle_medias_embeddings`

Query is running:   0%|          |

Downloading:   0%|          |

,uri,size,content_type,metadata,ref,mm_embedding
0,gs://vehicle_media/car-crash2.jpg,206916,image/jpeg,[],"{'uri': 'gs://vehicle_media/car-crash2.jpg', '...","[0.0101316115, 0.0513879471, 0.00523022516, -0..."
1,gs://vehicle_media/car-crash.jpeg,10346,image/jpeg,[],"{'uri': 'gs://vehicle_media/car-crash.jpeg', '...","[0.00324978982, 0.0341516435, -0.00709658302, ..."
2,gs://vehicle_media/car-crash-5.jpg,53572,image/jpeg,[],"{'uri': 'gs://vehicle_media/car-crash-5.jpg', ...","[-0.0102876304, 0.0701947585, -0.0132986931, -..."
3,gs://vehicle_media/67fdf83a892ab.image.jpg,22239,image/jpeg,[],{'uri': 'gs://vehicle_media/67fdf83a892ab.imag...,"[0.044124797, 0.0648579, -0.0285160542, -0.014..."
4,gs://vehicle_media/car-crash-1.jpeg,202572,image/jpeg,[],"{'uri': 'gs://vehicle_media/car-crash-1.jpeg',...","[0.0342633091, 0.0359671414, -0.0264450163, -0..."
5,gs://vehicle_media/car-crash-6.jpg,42350,image/jpeg,[],"{'uri': 'gs://vehicle_media/car-crash-6.jpg', ...","[-0.00403600326, 0.0177434459, 0.000888852926,..."


In [ ]:
%%bigquery --project {PROJECT_ID}

WITH query_image AS (
  SELECT
    uri AS query_uri,
    mm_embedding AS query_embedding
  FROM `claimwise.vehicle_medias_embeddings`
  WHERE uri = "gs://vehicle_media/car-crash-1.jpeg"
)

SELECT
  e.uri,
  e.content_type,
  ML.DISTANCE(e.mm_embedding, q.query_embedding, 'COSINE') AS similarity
FROM
  `claimwise.vehicle_medias_embeddings` e,
  query_image q
WHERE e.uri != q.query_uri
ORDER BY similarity ASC   -- smaller cosine distance = more similar
LIMIT 5;

Query is running:   0%|          |

Downloading:   0%|          |

,uri,content_type,similarity
0,gs://vehicle_media/car-crash.jpeg,image/jpeg,0.441971
1,gs://vehicle_media/car-crash-5.jpg,image/jpeg,0.446859
2,gs://vehicle_media/67fdf83a892ab.image.jpg,image/jpeg,0.462082
3,gs://vehicle_media/car-crash2.jpg,image/jpeg,0.507423
4,gs://vehicle_media/car-crash-6.jpg,image/jpeg,0.591253
